In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
df = pd.read_csv("/kaggle/input/mushroom-classification-enhanced/mushroom.csv")

pd.set_option("display.max_columns", None)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
df.isnull().sum(axis=0)

In [ ]:
for i in df.columns[1:]:
    df[i].fillna(df[i].mode()[0], inplace=True)

In [ ]:
df.isnull().sum(axis=0)

In [ ]:
index = 0

for i in range(4):
    fig, axes = plt.subplots(ncols=5, figsize=(15, 6))
    for j in range(5):
        count = df[df.columns[index]].value_counts()
        count.plot(kind="bar", ax=axes[j])
        for container in axes[j].containers:
            axes[j].bar_label(container)
        axes[j].set_yticklabels(())
        axes[j].set_ylabel("")
        axes[j].set_xlabel("")
        axes[j].set_title(df.columns[index])
        index += 1
    plt.tight_layout()
    plt.show()

fig, axes = plt.subplots(ncols=3, figsize=(15, 6))
for i in range(3):
    count = df[df.columns[index]].value_counts()
    count.plot(kind="bar", ax=axes[i])
    for container in axes[i].containers:
        axes[i].bar_label(container)
    axes[i].set_yticklabels(())
    axes[i].set_ylabel("")
    axes[i].set_xlabel("")
    axes[i].set_title(df.columns[index])
    index += 1
plt.tight_layout()
plt.show()

In [ ]:
le = LabelEncoder()

for i in df.columns:
    df[i] = le.fit_transform(df[i].values)

In [ ]:
scaler = MinMaxScaler()

x = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

x = scaler.fit_transform(x)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=0.2)


In [ ]:
forest = RandomForestClassifier()

forest.fit(x, y)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)

forest_importances = pd.Series(importances, index=df.columns[1:])

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

In [ ]:
def training():
    scores, reports, cms = [], dict(), dict()
    
    for i, j in zip(models, names):
        i.fit(x_train, y_train)
        pred = i.predict(x_test)
        score = accuracy_score(pred, y_test)*100
        report = classification_report(pred, y_test)
        cm = confusion_matrix(pred, y_test)
        scores += [score]
        reports[j] = report
        cms[j] = cm
        
    dd = pd.DataFrame({"score": scores}, index=names)
    dd = dd.sort_values("score", ascending=False)
    dd["score"] = round(dd["score"], 2)
    fig, axes = plt.subplots()
    dd["score"].plot(kind="bar", ax=axes)
    for container in axes.containers:
        axes.bar_label(container)
    axes.set_yticklabels(())
    axes.set_ylabel("")
    axes.set_xlabel("")
    plt.show()
    
    
    index = 0
    
    for i in range(2):
        fig, axes = plt.subplots(ncols=3, figsize=(15, 6))
        for j in range(3):
            sns.heatmap(cms[dd.index[index]], annot=True, ax=axes[j])
            axes[j].set_title("{}: {}%".format(dd.index[index], dd.iloc[index, 0]))
            index += 1
        plt.tight_layout()
        plt.show()
        
    for i in dd.index:
        print("*"*30)
        print(i)
        print(reports[i])
        print("\n\n")

In [ ]:
rfc = RandomForestClassifier()
abc = AdaBoostClassifier()
gbc = GradientBoostingClassifier()
etc = ExtraTreesClassifier()
xgb = XGBClassifier()
lgb = LGBMClassifier()

models = [rfc, abc, gbc,
         etc, xgb, lgb]

names = ["Random Forest", "Ada Boost", "Gradient Boosting",
        "Extra Trees", "XGBoost", "LightGBM"]

In [ ]:
training()
